# ***Muziek classificeren***

|Teamleden|Kaggle Username|GitHub Username|
|--|--|--|
|Busse Heemskerk|bussejheemskerk|BJHeemskerk|
|Declan van den Hoek|declanvdh|DeclanvandenHoek|
|Isa Dijkstra|isadijkstra|IsaD01|

In dit notebook gaan we kleine muziek samples classificeren met behulp van unsupervised learning. Een deel van deze bestand heeft een genre label, terwijl de meeste dit niet zullen hebben. Aan ons is de taak om zo accuraat mogelijk te bepalen welke genres de unlabeled samples hebben, door middel van Unsupervised Learning.

Voor het project hebben we gewerkt in [GitHub](https://github.com/BJHeemskerk/MachineLearning/tree/main/Muziek), om makkelijk de bestanden te delen. Van elk model zijn de voorspellingen ook geupload naar [Kaggle](https://www.kaggle.com/competitions/muziek-genre-clustering/overview).

## **Libaries en data inladen** <a name='h1'></a>

In [11]:
import os
import librosa as lr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from librosa.core import stft
from librosa.core import amplitude_to_db
import librosa.feature as lf
from librosa.feature import spectral_bandwidth, spectral_centroid

# Om audio af te kunnen spelen
from IPython.display import Audio

In de onderstaande cel, die gebaseerd is op de code van het Machine Learning notebook uit week 11, wordt de data ingelezen en in een dataframe gezet.

In [12]:
def mfccs(data, sfreq):
    """
    Bereken Mel Frequency Cepstral Coefficients
    (MFCCs) voor het gegeven geluidssignaal.

    Parameters:
    ----------
    data : array
        Het geluidssignaal.

    sfreq : int
        De samplefrequentie van het geluidssignaal.

    Returns:
    ----------
    datadict : dict
        Een dictionary met gemiddelde waarden
        van MFCCs per coëfficiënt.
    """
    # Toepassen van mfcc via librosa
    mfcc = lr.feature.mfcc(y=data, sr=sfreq)
    datadict = {}

    # Vullen van datadict
    for var in range(len(mfcc)):
        datadict[f'mfcc{var + 1}_mean'] = np.mean(mfcc[var, :])

    return datadict

def calculate_spectrograms(audio_clips, n_fft=2048, hop_length=512, win_length=None):
    """
    Bereken het spectrogram voor elk audiofragment in de audio_clips-reeks.

    Parameters:
    ----------
    audio_clips : lijst
        Een lijst met audioclips (numpy-arrays).
    n_fft : int, optioneel
        Het aantal datapunten dat wordt gebruikt in elk blok voor de FFT (standaard 2048).
    hop_length : int, optioneel
        Het aantal samples tussen opeenvolgende frames (standaard 512).
    win_length : int, optioneel
        De venstergrootte (standaard is `n_fft`).

    Returns:
    ----------
    spectrograms : lijst
        Een lijst met spectrogrammen die overeenkomen met elk audioclip.
    spec_db : array
        De spectrogrammen in decibels, voor het plotten.
    """
    spectrograms = []
    spectrograms_db = []

    for clip in audio_clips:

        # Calculate the STFT. Use lr.stft() here.
        stft_matrix = stft(y=clip,
                           n_fft=n_fft,
                           hop_length=hop_length,
                           win_length=win_length)

        # Calculate the magnitude of the STFT (spectrogram). Use np.abs() here.
        spectrogram = np.abs(stft_matrix)

        # Convert to decibels. Use amplitude_to_db() here.
        spec_db = amplitude_to_db(S=spectrogram,
                                  ref=np.max)

        # Append the spectrogram to the list
        spectrograms.append(spectrogram)

        # Append spectrogram in decibels to the list
        spectrograms_db.append(spec_db)

    return spectrograms, spectrograms_db


def calculate_spectral_features(spectrograms):
    """
    Bereken de centroid en bandbreedte voor elk spectrogram in een lijst.

    Parameters:
    ----------
    spectrograms : lijst
        Een lijst met spectrogrammen.

    Returns:
    ----------
    bandwidths : lijst
        Een lijst met bandbreedtes die overeenkomen met elk spectrogram.
    centroids : lijst
        Een lijst met centroids die overeenkomen met elk spectrogram.
    """
    bandwidths = []
    centroids = []

    for spectrogram in spectrograms:

        # Calculate the bandwidth for the spectrogram (use lr.feature.spectral_bandwidth)
        spec_bw = spectral_bandwidth(S=spectrogram)

        # Calculate the spectral centroid for the spectrogram (use lr.feature.spectral_centroid)
        spec_cn = spectral_centroid(S=spectrogram)

        # Append the spectral bandwidth to the list
        bandwidths.append(spec_bw)

        # Append the spectral centroid to the list
        centroids.append(spec_cn)

    return bandwidths, centroids

def calculate_spectral_contrast(data, sr, n_fft=2048, hop_length=512):
    """
    Bereken spectrale contrasten van het gegeven geluidssignaal.

    Parameters:
    ----------
    data : array
        Het geluidssignaal.
    sr : int
        De samplefrequentie van het geluidssignaal.
    n_fft : int, optioneel
        Grootte van het FFT-venster (standaard 2048).
    hop_length : int, optioneel
        Stapgrootte tussen raampunten (standaard 512).

    Returns:
    ----------
    spectral_contrast : array
        Spectrale contrasten.
    """
    spectral_contrast = lf.spectral_contrast(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)

    return spectral_contrast

def calculate_tonnetz(data, sr):
    """
    Bereken de tonnetz-functies van het gegeven geluidssignaal.

    Parameters:
    ----------
    data : array
        Het geluidssignaal.
    sr : int
        De samplefrequentie van het geluidssignaal.

    Returns:
    ----------
    tonnetz : array
        Tonnetz-functies.
    """
    tonnetz = lf.tonnetz(y=data, sr=sr)

    return tonnetz

def calculate_spectral_rolloff(data, sr, roll_percent=0.85, n_fft=2048, hop_length=512):
    """
    Bereken de spectrale rolloff van het gegeven geluidssignaal.

    Parameters:
    ----------
    data : array
        Het geluidssignaal.
    sr : int
        De samplefrequentie van het geluidssignaal.
    roll_percent : float, optioneel
        Percentage van de spectrale energie waar de rolloff wordt berekend (standaard 0.85).
    n_fft : int, optioneel
        Grootte van het FFT-venster (standaard 2048).
    hop_length : int, optioneel
        Stapgrootte tussen raampunten (standaard 512).

    Returns:
    ----------
    spectral_rolloff : array
        Spectrale rolloff.
    """
    spectral_rolloff = lf.spectral_rolloff(y=data, sr=sr, roll_percent=roll_percent, n_fft=n_fft, hop_length=hop_length)

    return spectral_rolloff


In [13]:
# Load labeled data from CSV
labeled_data = pd.read_csv("labels_new.csv", sep=',')
labeled_data = labeled_data.sort_values('filename')

# Map pakken met de juiste samples
base_dir = "labeled"

# Aanmaken lists voor data
audio_data = []
sample_freqs = []
mfcc_data = {}

# Lengte is 30 sec op 22050Hz
lengte = 30 * 22050

# Process each audio file
for file in os.listdir(base_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(base_dir, file)
        data, sfreq = lr.load(file_path, sr=None)

        # Truncate or pad the audio
        if len(data) > lengte:
            # Truncate the data
            data = data[:lengte]
        elif len(data) < lengte:
            # Pad with zeros
            padding = lengte - len(data)
            data = np.pad(data, (0, padding), mode='constant')

        # Append the processed data and label
        audio_data.append(data)
        sample_freqs.append(sfreq)

        # Make the mfcc data using your custom function
        mfcc_dict = mfccs(data, sfreq)

        # Update the dictionary with individual MFCC coefficients
        for key, value in mfcc_dict.items():
            if key not in mfcc_data:
                mfcc_data[key] = []
            mfcc_data[key].append(value)

# Convert to numpy array
audio_data = np.stack(audio_data, axis=0)
sample_freqs = np.array(sample_freqs)

# Create a DataFrame from the processed audio data
audio_df = pd.DataFrame({
    'filename': os.listdir(base_dir),
    'data': audio_data.tolist(),
    'Hz': sample_freqs.tolist(),
})

# Add columns for each MFCC coefficient
for key, values in mfcc_data.items():
    audio_df[key] = values

# Merge the labeled_data DataFrame with the audio_df DataFrame based on the 'filename' column
df = labeled_data.merge(audio_df, how='left', on='filename')

# Calculate spectrograms
spectrograms, spectrograms_db = calculate_spectrograms(audio_data, n_fft=2048, hop_length=512, win_length=None)
bandwidths, centroids = calculate_spectral_features(spectrograms)
spectral_contrast = calculate_spectral_contrast(audio_data, sfreq, n_fft=2048, hop_length=512)
tonnetz = calculate_tonnetz(audio_data, sfreq)
spectral_rolloff = calculate_spectral_rolloff(audio_data, sfreq, roll_percent=0.85, n_fft=2048, hop_length=512)

# Adding these columns to the dataframe
df['mean_bandwidth'] = [np.mean(arr) for arr in bandwidths]
df['mean_centroids'] = [np.mean(arr) for arr in centroids]
df['mean_spectral_contrast'] = [np.mean(arr) for arr in spectral_contrast]
df['mean_tonnetz'] = [np.mean(arr) for arr in tonnetz]
df['mean_spectral_rolloff'] = [np.mean(arr) for arr in spectral_rolloff]

# Display the merged DataFrame
display(df.head())

,filename,genre,data,Hz,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,...,mfcc16_mean,mfcc17_mean,mfcc18_mean,mfcc19_mean,mfcc20_mean,mean_bandwidth,mean_centroids,mean_spectral_contrast,mean_tonnetz,mean_spectral_rolloff
0,m00002.wav,jazz,"[-0.016357421875, -0.0228271484375, -0.0146789...",22050,-298.807953,112.078209,6.485770,28.386517,-6.764679,16.651894,...,13.479482,9.419415,6.914652,7.877785,-1.782740,1919.917650,1451.498371,24.225544,0.000351,3046.089914
1,m00039.wav,reggae,"[-0.09478759765625, -0.15338134765625, -0.1439...",22050,-169.243668,110.447716,-8.553957,43.898693,0.266454,26.646509,...,1.102364,-4.261436,4.327076,-3.458247,1.208493,2019.252686,1811.358216,22.132186,0.016613,3854.901690
2,m00041.wav,pop,"[0.078033447265625, -0.03765869140625, 0.12664...",22050,-18.854591,71.328522,-3.743232,-1.396592,0.710347,-1.049137,...,-3.588230,0.891752,-0.496282,0.708363,1.672521,2992.192112,3111.061099,17.239507,-0.035761,6745.275879
3,m00072.wav,disco,"[0.1060791015625, 0.0849609375, 0.062103271484...",22050,-69.599335,83.059570,-16.599524,0.119469,7.415704,0.769619,...,-1.885092,4.043784,3.654379,1.913874,3.608692,2709.990169,2625.095044,19.452643,-0.018575,5606.407765
4,m00096.wav,disco,"[-0.03607177734375, -0.105682373046875, -0.201...",22050,-91.886307,87.604057,-2.058175,34.285538,-18.153370,19.344702,...,-1.051909,-11.649620,0.293441,-1.279737,0.098982,2486.020650,2550.135384,21.882917,0.009210,5585.291227


Nu de data is ingeladen kunnen we met behulp van de Audio functie een bestand afspelen in het notebook.

In [3]:
# Pick a random audio clip
random_index = np.random.choice(len(df), size=1, replace=False).item()

# Access the data, Hz, filename, and genre
clip = np.array(df.at[random_index, 'data'])
sfreq = df.at[random_index, 'Hz']
file = df.at[random_index, 'filename']
genre = df.at[random_index, 'genre']

# Print the name and genre
print(f"File and genre: {file}, {genre}")

# Play the clip
Audio(data=clip, rate=sfreq)


File and genre: m00637.wav, hiphop


## **Feature Engineering** <a name='h2'></a>

In [4]:
test_file = "labeled/m00002.wav"
y, sr = lr.load(test_file, sr=None)  

### Root Mean Square energy

De energie van muziek kan ook wel gedefinieerd worden als de intensiteit van de muziek. In praktische termen is dit de amplitude van de golf (?)(Ik breidt het wel uit en zoek er een bron bij)

Dit geeft ons een idee van de algemene intensiteit van een nummer. Theoretisch zou dit kunnen helpen met het onderscheiden tussen genres, een metal nummer zou over het algemeen intenser zijn dan een klassiek stuk. 

In [15]:
def rms_energy_features(data, sfreq):
    """
    Berekent de RMS Energy voor het ingegeven audiobestand.
    
    Parameters:
    data : array
        Het geluidssignaal.

    sfreq : int
        De samplefrequentie van het geluidssignaal.

    Returns:
    ----------
    datadict : dict
        Een dictionary met gemiddelde waarden
        van MFCCs per coëfficiënt.
    """
    # Compute RMS Energy
    rms_energy = librosa.feature.rms(y=data)[0]

    datadict = {}

    # Fill datadict
    datadict['rms_energy_mean'] = np.mean(rms_energy)

    return datadict

### Chroma Feature

In muziektermen is de chroma feature gerelateerd aan de 12 toon klassen. Hiermee kunnen we een indicatie geven van de toonhoogtes van een muziekfragment. Dit hebben we gekozen omdate verschillende genres muziek over het algemeen ook verschillende tonen gebruiken.

In [16]:
def chroma_features(data, sfreq):
    # Compute Chroma feature
    chroma = librosa.feature.chroma_stft(y=data, sr=sfreq)

    datadict = {}

    # Fill datadict
    for var in range(len(chroma)):
        datadict[f'chroma{var + 1}_mean'] = np.mean(chroma[var, :])

    return datadict

### Zero Crossing Rate

De zero crossing rate houd in hoe vaak het (audio)signaal verandert van positief naar negatief of andersom. Hiermee is grofweg te zien hoeveel ruis er in een fragment zit. 

In [17]:
def zero_crossing_rate_features(data, sfreq):
    # Compute Zero Crossing Rate (ZCR)
    zcr = librosa.feature.zero_crossing_rate(y=data)

    datadict = {}

    # Fill datadict
    datadict['zcr_mean'] = np.mean(zcr)

    return datadict